# Piano to Sheet

Converting .wav piano pieces into music sheets.

I

## Pipeline break down ( beta)

1. Loading in the wave file,
2. 



## Audio Processing  Libraries

For the processing libraries, we will be using pretty_midi to extract out label information from the given midi files.  
And for processing our wav audio files. We will be using librosa.

## Machine Learning Lib

In [ ]:
pip uninstall tensorflow

In [ ]:
import numpy as np
import matplotlib as plt
import tensorflow as tf
import sklearn
import librosa 
import json
import csv
import pandas as pd
import pretty_midi as pm
import mido
import IPython.display as ipd

# Model Saving funtion and Model loading function

Having a model saving function and a modle loading function in which we can train the model in small epoch progress.

In [ ]:
def modelSaving(model, version):
    path = f"./Models/{version}"
    print("Most recent saved model version is at ")
    print(path)
    joblib.dump(model, path)
    print(f"Model successfully saved")    
    

In [ ]:
def modelLoading(file):
    #file is a file path that reside in the directory of models
    
    saveTo = joblib.load(file)
    print(f"Model successfully loaded from {file}")
    return saveTo
    

# Preprocessing the data

In [ ]:

labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
with open(labels_file_path, 'r') as file:
    data = json.load(file)

all_sets = {}
all_sets['train'] = []
all_sets['validation'] = []
all_sets['test'] = []

def sortingsets (data, allsets):
    for key in data:
        if data[key] == 'train':
            all_sets['train'].append(key)
        elif data[key] == 'validation':
            all_sets['validation'].append(key)
        else:
            all_sets['test'].append(key)

def save_index_to_csv(all_sets):
    for key in all_sets:
    
        path = f"traindata/maestro-v3.0.0-midi/maestro-v3.0.0/{key}_indicies.csv"
        df = pd.DataFrame({'Index': all_sets[key]})
        df.to_csv(path, index=False)

def load_index_from_csv(path):
    indices = []
    with open(path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            indices.append(int(row[0]))
    return indices





traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv  
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/test_indicies.csv   
traindata/maestro-v3.0.0-midi/maestro-v3.0.0/validation_indicies.csv

In [ ]:
#save_index_to_csv(all_sets)

### Data Preprocessing
implenting utility functions such as the randomizing the data set for training.

In [ ]:
### making the random seed 
np.random.seed(40)

In [ ]:


def randomizeing(data_set):
    ds = np.array(data_set)
    np.random.shuffle(ds)
    return ds

    
def select_N_randomized_from_set(n, data_set):
    nparry = randomizeing(data_set)
    return nparry[:n]


## function loading in the wav function
def load_wav_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    
    return librosa.load(wav_path, sr=None)

## function loading in the midi function
def load_midi_from_index(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    midi_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/" + data['midi_filename'][str(index)]
    return pm.PrettyMIDI(midi_path)
    
## showing the audio of the wave
def show_wav(index):
    labels_file_path = "traindata/maestro-v3.0.0-midi/maestro-v3.0.0/maestro-v3.0.0.json"
    with open(labels_file_path, 'r') as file:
        data = json.load(file)
    wav_path = "traindata/maestro-v3.0.0/maestro-v3.0.0/" + data['audio_filename'][str(index)]
    return wav_path

## loading in the wav and midi pair
def load_wav_midi_pair(index): ## (wav, midi)
    return load_wav_from_index(index), load_midi_from_index(index)

In [ ]:
train_data = load_index_from_csv("traindata/maestro-v3.0.0-midi/maestro-v3.0.0/train_indicies.csv")
print(len(train_data))
subset_train_data = select_N_randomized_from_set(50, train_data)

print(len(subset_train_data))

In [ ]:
(wav, sr), midi = load_wav_midi_pair(1025)
# midi = load_midi_from_index(1025)


In [ ]:
ipd.Audio(filename=show_wav(1025))

start_time = 0.0  # Start at 5 seconds
end_time = 1.0   # End at 10 seconds

# Step 3: Convert time to sample indices
start_sample = int(start_time * sr)
end_sample = int(end_time * sr)

# Step 4: Slice the audio array
audio_segment = wav[start_sample:end_sample]
ipd.Audio(audio_segment, rate= sr)

In [ ]:
ipd.Audio(filename=show_wav(505))

In [ ]:
ballade1, sr = load_wav_from_index(505)

noised_ballade1 = add_gaussian_noise(ballade1)

In [ ]:
ipd.Audio(data= ballade1, rate= sr)

In [ ]:
ipd.Audio(data= noised_ballade1, rate = sr)

### Audio Preprocessing functions

In [ ]:
def add_gaussian_noise(audio, noise_level=0.0006):
    noise = np.random.normal(0, noise_level, audio.shape)
    return audio + noise



## This function extracts all played notes in the midi Object, which it will be futher trained with the aligne ed wave object
def extract_midi_notes(midi):
    notes = []
    for instrument in midi.instruments:
        for note in instrument.notes:
            notes.append({
                'pitch': note.pitch,
                'start': note.start,
                'end': note.end,
                'velocity': note.velocity
            })
    return notes


def frame_aligning_wav(t: int, jump_len: int, frame_len: int, wav):
    begin = t * jump_len
    return wav[begin: begin + frame_len]

def frame_aligning_midi(t: int, jump_len: int, frame_len: int, midi): 
    begin = t * jump_len
    ## Perform binary Search to find the midi object

    left = midi
    right = 

### Calculating the added noise level to the wav function.

In [ ]:
def compute_noise_power(audio, snr_dB):
    """
    Compute the noise power needed for a given SNR in dB.
    :param audio: NumPy array of the audio signal.
    :param snr_dB: Desired Signal-to-Noise Ratio in dB.
    :return: Noise power.
    """
    # Calculate signal power (mean squared amplitude)
    signal_power = np.mean(audio ** 2)
    
    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr_dB / 10)
    
    # Calculate noise power
    noise_power = signal_power / snr_linear
    return noise_power

div = 100
somewav = select_N_randomized_from_set(div, train_data)
print(len(somewav))
summing = 0 #summing the proper noise level
for i in range(10):
    wav, _ = load_wav_from_index(somewav[i])
    summing += np.sqrt(compute_noise_power(wav, 20)) ## desired training SNR to be 15 dB

print(f"Our desired training noise is about {summing/div}, setting it to be our default added noise to wav")